### Naludaq Version
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Prerequisites
+ Acquisition captured previously (can be from NaluScope)

In [ ]:
# Run this code for plotting
import matplotlib.pyplot as plt

from naluexamples.helpers.plotting import get_color_mapping, set_plot_style

def plot_event(event: dict, channels: list[int]=None, title: str=None):
    """Plot an event dict.

    Args:
        event (dict): the parsed event
        channels (list[int]): Channels to plot. Defaults to all channels.
        title (str): Optional title for the plot
    """
    channels = channels or range(len(event["data"]))
    title = title or f"Event #{event.get('pkg_num', '?')}"


    # Plot the event
    plt.figure(figsize=(8,8), constrained_layout=True)
    plt.xlabel("Time")
    plt.ylabel("ADC Counts")
    plt.title(title)
    cmap = get_color_mapping("ocean")
    set_plot_style(font_size=18, font_family="monospace")
    for i, channel in enumerate(channels):
        time = event["time"][channel]
        data = event["data"][channel]
        plt.plot(time, data, ".-", label=f"Channel {channel}", color=cmap(i / len(channels)))
    plt.legend()
    plt.show()

## Legacy Acquisitions

Acquisitions captured in the older "legacy" format are gzipped Python pickles. 
The data structure is a bit different than the newer format, and the entire acquisition must be loaded into memory.

In [ ]:
from naludaq.io import load_pickle_acquisition

ACQ_PATH = r"<PATH TO .acq FILE>"

# Load the acquisition from disk
ACQ = load_pickle_acquisition(ACQ_PATH)

print("Board model:", ACQ.params["model"])
print("Number of events:", len(ACQ))
print("Pedestals stored:", ACQ.pedestals is not None)

Accessing events is different with legacy acquisitions, as they must be manually parsed.

In [ ]:
from naludaq.parsers import get_parser

EVENT_INDEX = 0

parser = get_parser(ACQ.params)
event = parser.parse(ACQ[EVENT_INDEX])
plot_event(event)

## Pedestals Subtraction

For both acquisition formats, the events stored in the acquisitions are not pedestal-subtracted. To obtain pedestals-subtracted events, the process must be performed manually on the events. The example below uses the new acquisition format, but the same process applies to legacy acquisitions.

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter


# Fetch an event, pedestals, and params from disk
ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"
EVENT_INDEX = 0

ACQ = load_pickle_acquisition(ACQ_PATH)

event = ACQ[EVENT_INDEX]
params = ACQ.params
pedestals = ACQ.pedestals

if pedestals is None:
    print("Acquisition must have pedestals in order to correct data!")
else:
    # Apply correction (in place)
    corrector = PedestalsCorrecter(params, pedestals)
    corrected_event = corrector.run(event, correct_in_place=True)
    plot_event(corrected_event)